## Install

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# install bark (make sure you have torch>=2 for much faster flash-attention)
!pip install git+https://github.com/suno-ai/bark.git

  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-j9qkiel1
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-j9qkiel1
  Resolved https://github.com/suno-ai/bark.git to commit 773624d26db84278a55aacae9a16d7b25fbccab8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 51.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 124.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 82.1 MB/s eta 0:00:00
  

In [5]:
!pip install pydub

In [6]:
%cd /content/drive/MyDrive/TuChisteDiario

/content/drive/MyDrive/TuChisteDiario


In [7]:
!pip install SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 44.2 MB/s eta 0:00:00


In [8]:
!pip install -U git+https://github.com/jianfch/stable-ts.git

  Cloning https://github.com/jianfch/stable-ts.git to /tmp/pip-req-build-8yrtsp_y
  Running command git clone --filter=blob:none --quiet https://github.com/jianfch/stable-ts.git /tmp/pip-req-build-8yrtsp_y
  Resolved https://github.com/jianfch/stable-ts.git to commit 3dfbd722a8edb606a2d819fee49ff9c5db4bf0f2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.3/794.3 kB 16.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.1/153.1 kB 19.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is

## Read Chistes

In [9]:
import pandas as pd

In [10]:
df_chistes = pd.read_csv('chistes_con_metadatos.csv')

## Long text generator

In [11]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


from IPython.display import Audio
import nltk  # we'll use this to split into sentences
import numpy as np

from bark.generation import (
    generate_text_semantic,
    preload_models,
)
from bark.api import semantic_to_waveform
from bark import generate_audio, SAMPLE_RATE

nltk.download('punkt')

from scipy.io.wavfile import write
from pydub import AudioSegment
from pydub import effects

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [12]:
preload_models()

Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th
100%|██████████| 88.9M/88.9M [00:00<00:00, 113MB/s]


In [13]:
import stable_whisper
model = stable_whisper.load_model('medium')

100%|█████████████████████████████████████| 1.42G/1.42G [00:25<00:00, 59.7MiB/s]


In [14]:
def agrupar_subtitulos(nombre_archivo, nombre_salida, num_palabras):
    try:
        with open(nombre_archivo, 'r', encoding='utf-8') as archivo:
            lineas = archivo.readlines()

            subtitulos = []
            tiempos = []
            texto_subtitulo = ""
            tiempo_actual = ""

            for linea in lineas:
                linea = linea.strip()
                if not linea:
                    subtitulos.append(texto_subtitulo.strip())
                    tiempos.append(tiempo_actual)
                    texto_subtitulo = ""
                    tiempo_actual = ""
                elif "-->" in linea:
                    tiempo_actual = linea
                elif not linea.isdigit():
                    texto_subtitulo += linea + " "

            subtitulos.append(texto_subtitulo.strip())
            tiempos.append(tiempo_actual)

            subtitulos_agrupados = []
            tiempos_agrupados = []
            texto_agrupado = ""
            contador_palabras = 0
            tiempo_inicio = tiempos[0].split(" --> ")[0]
            for i, subtitulo in enumerate(subtitulos):
                palabras = subtitulo.split()
                contador_palabras += len(palabras)
                texto_agrupado += subtitulo + " "
                if contador_palabras >= num_palabras or i == len(subtitulos) - 1:
                    tiempo_fin = tiempos[i].split(" --> ")[1]
                    subtitulos_agrupados.append(texto_agrupado.strip())
                    tiempos_agrupados.append(tiempo_inicio + " --> " + tiempo_fin)
                    texto_agrupado = ""
                    contador_palabras = 0
                    if i+1 < len(tiempos):
                        tiempo_inicio = tiempos[i+1].split(" --> ")[0]

            with open(nombre_salida, 'w', encoding='utf-8') as salida:
                for i, (tiempo, subtitulo) in enumerate(zip(tiempos_agrupados, subtitulos_agrupados)):
                    salida.write(f"{i}\n")
                    salida.write(tiempo + "\n")
                    salida.write(subtitulo + "\n\n")

    except FileNotFoundError:
        print(f"No se pudo encontrar el archivo {nombre_archivo}")
    except Exception as e:
        print(f"Se produjo un error: {e}")


In [15]:
df_chistes.loc[83:90,'audio_creado']

83    True
84    True
85    True
86    True
87    True
88    True
89    True
90    True
Name: audio_creado, dtype: bool

In [16]:
# Risa enlatada
risa_enlatada = AudioSegment.from_wav("risas.wav")
# Chistes a crear
df_chistes_crear = df_chistes[~df_chistes['audio_creado']]

for index, row in df_chistes_crear.iterrows():

  print(f"-----------------------------")
  print(f"------ Chiste {index} -------")
  print(f"-----------------------------")

  text_prompt = row['text']
  print(f"Chiste: {text_prompt}")

  #creo carpeta del chiste
  path = f"audios_chistes/chiste_{index}"
  if os.path.exists(path):
    !rm -r {path}
  os.makedirs(path)
  print('100%|██████████| carpeta creada')

  for genero in ['masc','fem']:

    print(f"-> Chiste {index} {genero}")

    text_prompt = row['text']
    text_prompt = text_prompt.replace("...","").replace("- ","-")
    script = text_prompt.replace("\n", " ").strip()
    sentences = nltk.sent_tokenize(script)

    SPEAKER = "es_speaker_7" if genero == 'masc' else "v2/es_speaker_9"
    NAME_AUDIO = "/audio_" + genero
    NAME_SRT = "/subs_" + genero
    silence = np.zeros(int(0.2 * SAMPLE_RATE))  # quarter second of silence

    pieces = []
    for sentence in sentences:
        audio_array = generate_audio(sentence, history_prompt=SPEAKER)
        pieces += [audio_array, silence.copy()]

    audio = np.concatenate(pieces)
    # Escalar y convertir el array a formato entero de 16 bits
    scaled_audio = np.int16(audio * 32767.0)
    ruta_resultado = path + "/temp_chiste"
    write(ruta_resultado, SAMPLE_RATE, scaled_audio)

    #Aumentar velocidad
    velocidad_X = 1.2 # No puede estar por debajo de 1.0
    sound = AudioSegment.from_file(ruta_resultado)
    os.remove(ruta_resultado)
    so = sound.speedup(velocidad_X, 150, 25)
    ruta_resultado = path + "/temp_chiste_speedup"
    so.export(ruta_resultado, format="wav")
    print(f"100%|██████████| velocidad {velocidad_X}")

    # Crear Subtitulos
    result = model.transcribe(ruta_resultado)
    result.to_srt_vtt(path + NAME_SRT + '_words.srt', segment_level=False, word_level=True)
    agrupar_subtitulos(path + NAME_SRT + '_words.srt',path + NAME_SRT + '.srt', 4)
    print('100%|██████████| subtítulos creados')

    # Crear Risas
    audio_existente = AudioSegment.from_wav(ruta_resultado)
    os.remove(ruta_resultado)
    audio_final = audio_existente + risa_enlatada
    ruta_resultado = path + NAME_AUDIO
    audio_final.export(ruta_resultado, format="wav")
    print('100%|██████████| risas concatenadas')

  #marcamos chiste como creado
  df_chistes.loc[index,'audio_creado'] = True
  df_chistes.to_csv('chistes_con_metadatos.csv',index=False)

-----------------------------
------ Chiste 91 -------
-----------------------------
Chiste: - Nos gusta mucho su currículum, señor Juan, pero no especifica su teléfono. - No. - ¿Y cómo le llamamos?  - Juan.
100%|██████████| carpeta creada
-> Chiste 91 masc


100%|██████████| 4/4 [00:04<00:00,  1.05s/it]


100%|██████████| velocidad 1.2


Transcribe:   0%|          | 0/9.03 [00:00<?, ?sec/s]

Detected language: spanish


...

Detected language: spanish


Transcribe: 100%|██████████| 6.04/6.04 [00:01<00:00,  3.38sec/s]


Saved: /content/drive/MyDrive/TuChisteDiario/audios_chistes/chiste_108/subs_fem_words.srt
100%|██████████| subtítulos creados
100%|██████████| risas concatenadas
-----------------------------
------ Chiste 109 -------
-----------------------------
Chiste: En clase dos niños: - Mi bolígrafo escribe en diez colores. - Bah! Eso no es nada! El mío los escribe todos! - Anda ya, pero si el tuyo sólo tiene tinta azul.. -Ya, pero es capaz de escribirlos todos te lo voy a demostrar ..Y escribe en una libreta : rojo, rosa, verde, amarillo, violeta, azul, granate, negro...
100%|██████████| carpeta creada
-> Chiste 109 masc


  3%|▎         | 1/33 [00:01<00:43,  1.37s/it]


KeyboardInterrupt: ignored